In [1]:
import pandas as pd
import json
import psycopg2
import random
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import string


###Connect to database. Get tweets with geometry

In [2]:
conn_dict = {'dbname':'zipfiantwitter', 'user':'clwilloughby', 'password': '', 'host':'/tmp'}
conn = psycopg2.connect(dbname=conn_dict['dbname'], user=conn_dict['user'], host='/tmp')
cur = conn.cursor()

map_query = """
SELECT timestamp_ms, text, geom
FROM tweeted
                ;"""

df_tweet_shapes = pd.read_sql(map_query, conn)

In [3]:
df_tweet_shapes = df_tweet_shapes.dropna()

In [4]:
print df_tweet_shapes.shape

(6963, 3)


In [31]:
number_of_unique_times = df_tweet_shapes.timestamp_ms.unique().shape
number_of_unique_geom = df_tweet_shapes.geom.unique().shape
print number_of_unique_times, number_of_unique_geom

(2320,) (1638,)


In [13]:
df_tweet_shapes.columns

Index([u'timestamp_ms', u'text', u'geom'], dtype='object')

#MiniExample of How I am processing Tweets:

In [109]:
#this gets me a count AND it gets me a 
def tokenize_tweets(text):
    '''Called by sql_tweets_sum_and_tokenized. Tokenizes a sting.'''
    punct = string.punctuation
    stop_words = stopwords.words('english')
    text_list = []
    snowball = SnowballStemmer('english')
    for word in text.lower().split():
        word = word.strip(punct)
        if word not in stop_words:
            text_list.append(snowball.stem(word))
    return text_list

def sql_tweets_sum_and_tokenized(df, timeseries= False):
    '''Takes a sql dataframe of tweets, geoids, and timestamps.
    Tokenizes the tweets. Groupsby geoid and timestamp (if True) to create overal counts
    Returns a new dataframe.'''
    #take the string and tokenize words for a list 
    df['text'] = df.text.apply(custom_stemmer)
    #create a column of ones for quick sum column during groupby
    df.insert(1, 'count', 1)
    #groupby parameters. Counts Tweets and appends tweet tokens to a master list. 
    if timeseries == False:
        df_group = df.groupby('geom').apply(lambda x: x.sum())
        #drop redundant columns
        df_group.drop('geom', 1, inplace=True)
        df_group.drop('timestamp_ms', 1, inplace=True)
    else:
        #somethign about time. This will be difficult. 
    return df_group

In [110]:
df_test2 = pd.DataFrame({'timestamp_ms':['124432','142352','345345','2535262','65767'], 'geom':[2,3,3,5,3], 'text':['wow that way cool', 'what is the deal?', 'i know what you are bt what am I', '', 'this is anotehr example']}) 

In [111]:
df_test2.head()

,geom,text,timestamp_ms
0,2,wow that way cool,124432
1,3,what is the deal?,142352
2,3,i know what you are bt what am I,345345
3,5,,2535262
4,3,this is anotehr example,65767


In [112]:
#run the mini function
sql_tweets_sum_and_tokenized(df_test2)

,count,text
geom,,
2,1,"[wow, way, cool]"
3,3,"[deal, know, bt, anotehr, exampl]"
5,1,[]
